<a href="https://colab.research.google.com/github/JiaxinLi-lipluszn/Temp_notebooks/blob/main/Supervised_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import 

In [ ]:
import os
import sys
import argparse

import torch
from torch import nn
import torch.distributed as dist
import torch.backends.cudnn as cudnn
from torchvision import datasets
from torchvision import transforms as pth_transforms
from torchvision import models as torchvision_models
import pandas as pd
from torch.utils.data import Dataset
from functools import partial
import numpy as np
from torch import optim
from pathlib import Path
from torch.autograd import Variable

import os
import argparse
import os
import sys
import datetime
import time
import math
import json
from pathlib import Path

import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.distributed as dist
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
from torch import optim
from torchvision import datasets, transforms
from torchvision import models as torchvision_models
import math
from functools import partial

import torch
import torch.nn as nn


import torch.nn.functional as F
from torch import optim
from torch.autograd import Variable

# Import utils

In [ ]:
os.chdir("/content/drive/MyDrive/scRNA/128+128_10subsets")

In [ ]:
import utils
import vision_transformer as vits_new
from utils import trunc_normal_

# Set paths

In [ ]:
data_path = "/content/drive/MyDrive/scRNA/pseudo_data/train"

# Helpers

## Plot gradient

In [ ]:
# To visualize the gradient flow
from matplotlib import pyplot as plt
def plot_grad_flow(named_parameters):
    ave_grads = []
    layers = []
    for n, p in named_parameters:
        if(p.requires_grad) and ("bias" not in n):
            layers.append(n)
            ave_grads.append(p.grad.abs().mean())
    plt.plot(ave_grads, alpha=0.3, color="b")
    plt.hlines(0, 0, len(ave_grads)+1, linewidth=1, color="k" )
    plt.xticks(range(0,len(ave_grads), 1), layers, rotation="vertical")
    plt.xlim(xmin=0, xmax=len(ave_grads))
    plt.xlabel("Layers")
    plt.ylabel("average gradient")
    plt.title("Gradient flow")
    plt.grid(True)

# Load data

## The old dataset class

In [ ]:
class scRNACSV(Dataset):
  def __init__(self, expression_matrix_path, perturb_list, transform = None, target_transform=None):
    self.expr = pd.DataFrame()
    self.samples = []
    self.labels = []
    for number_lab, label in zip(range(len(perturb_list)),perturb_list):
      path = expression_matrix_path + "/" + label + ".csv"
      df = pd.read_csv(path, index_col = 0)
      self.expr = pd.merge(self.expr, df, how = "outer", left_index = True, right_index = True)
      self.samples = self.samples + list(df.columns)
      lb = [number_lab for i in range(len(list(df.columns)))]
      self.labels = self.labels + lb
      print(f"This is label {label}, the number_lab is {number_lab}, there are {len(list(df.columns))} cells in this class")
    self.transform = transform
    self.target_transform = target_transform
    self.samples = [(self.samples[i],self.labels[i]) for i in range(len(self.labels))]
  def __len__(self):
    return self.expr.shape[1]

  def __getitem__(self,idx):
    one_cell = torch.from_numpy(np.array(self.expr.iloc[:,idx]))
    ret = self.transform(one_cell)
    return ret, self.labels[idx]
    #return one_cell, self.labels[idx]

## The new dataset class

In [ ]:
class scRNACSV(Dataset):
  def __init__(self, expr, meta, transform = None, target_transform=None):
    self.expr = expr
    self.meta = meta
    self.cells = list(expr.columns)
    self.labels = list(meta["perturb"])
    self.samples = [(self.cells,self.labels[i]) for i in range(len(self.labels))]
    self.label_keys = list(set(self.labels))
    self.label_keys.sort()
    self.label_dic = {}
    for label, i in zip(self.label_keys, range(len(self.label_keys))):
      self.label_dic[label] = i
    print(f"This is the label dictionary of this dataset {self.label_dic}")
    self.transform = transform
    self.target_transform = target_transform
  def __len__(self):
    return self.expr.shape[1]

  def __getitem__(self,idx):
    one_cell = torch.from_numpy(np.array(self.expr.iloc[:,idx]))
    if self.transform:
      ret = self.transform(one_cell)
      #print("Use transform")
    else:
      ret = one_cell
    lab = self.label_dic[self.labels[idx]]
    #print(f"This is the label {lab}")

    return ret, self.label_dic[self.labels[idx]]

## Prepare the single cell dataset's expr and meta

In [ ]:
!gunzip /content/drive/MyDrive/scRNA/data/rna_withlabels.csv.gz

In [ ]:
newdata = pd.read_csv("/content/drive/MyDrive/scRNA/data/rna_withlabels.csv", index_col=0)

In [ ]:
newdata

,IGKC,HBA2,HBB,HBA1,IGHA1,IGLC2,JCHAIN,HBM,IGHG1,IGHM,HBD,IGHG3,IGHG2,CA1,PTGDS,IGLC3,MZB1,AHSP,ELANE,HIST1H4C,IGLL1,PRTN3,CA2,MPO,DNTT,HBG2,AZU1,GYPA,PRDX2,HBG1,IGLL5,IGHGP,IGHA2,FAM178B,IGHG4,LYPD2,VPREB1,UBE2C,AL928768.3,APOC1,...,ARPC1A,FAM46C,SVOPL,TRAV5,TMEM183A,RPL3,LINC00570,BRIX1,DNAJC2,TRBV21-1,LINC00114,RP11-556I14.1,LECT1,SLC16A8,CRIP1,SEL1L3,POLR3H,AMT,SERPINB1,RNF26,RP11-656D10.3,RBP4,MTIF2,INF2,RP11-693J15.5,NAE1,KDELR2,TCEAL2,SPATS2L,INSR,RAD23B,COMMD4,PPARA,PFDN6,PDSS1,BANF1,DDI2,DCAF6,HSPA5,celltype.l1.5
0,-0.428204,0.183748,0.406723,-0.488968,1.248214,-0.614123,-0.324617,-0.181656,-0.200979,-0.414643,2.407124,-0.195287,-0.117689,-0.271599,-0.110263,-0.433555,-0.246179,-0.249721,-0.148742,-0.474248,1.514113,-0.12555,-0.180029,-0.226363,-0.08094,-0.046267,-0.167078,-0.131427,1.081326,-0.054701,-0.067024,-0.043884,-0.074466,-0.159364,-0.17881,-0.069823,-0.10839,2.493712,-0.137414,-0.160571,...,-0.317149,-0.177586,-0.075767,-0.03296,-0.333318,0.739594,-0.04443,-0.283894,2.660236,-0.017006,-0.014913,-0.014496,-0.016377,-0.015401,-0.471127,-0.241413,-0.178534,5.793597,2.086319,-0.183333,-0.021262,-0.021726,-0.159574,-0.14908,-0.154573,-0.324250,-0.445851,-0.038271,-0.117267,-0.216175,2.892558,-0.237605,-0.122832,-0.329040,-0.130763,-0.506981,3.164261,-0.197585,-0.44033,Progenitor cells
1,-1.047339,-0.601956,-0.985002,-0.488968,-0.552748,-0.614123,-0.324617,-0.181656,4.964708,-0.414643,-0.240882,-0.195287,-0.117689,-0.271599,-0.110263,-0.433555,-0.246179,-0.249721,-0.148742,-0.474248,-0.208137,-0.12555,6.346507,-0.226363,-0.08094,-0.046267,-0.167078,-0.131427,-0.523544,-0.054701,-0.067024,-0.043884,-0.074466,-0.159364,-0.17881,-0.069823,-0.10839,-0.175991,-0.137414,-0.160571,...,-0.317149,-0.177586,-0.075767,-0.03296,-0.333318,1.651260,-0.04443,-0.283894,-0.302578,-0.017006,-0.014913,-0.014496,-0.016377,-0.015401,-0.471127,-0.241413,-0.178534,-0.106664,-0.585515,-0.183333,-0.021262,-0.021726,-0.159574,-0.14908,-0.154573,-0.324250,-0.445851,-0.038271,-0.117267,-0.216175,-0.342985,-0.237605,-0.122832,-0.329040,-0.130763,-0.506981,-0.171434,-0.197585,-0.44033,gdT
2,-1.047339,-0.601956,1.534527,-0.488968,-0.552748,-0.614123,-0.324617,-0.181656,-0.200979,-0.414643,-0.240882,-0.195287,-0.117689,-0.271599,-0.110263,-0.433555,-0.246179,-0.249721,-0.148742,-0.474248,-0.208137,-0.12555,-0.180029,-0.226363,-0.08094,-0.046267,-0.167078,-0.131427,2.330472,-0.054701,-0.067024,-0.043884,-0.074466,-0.159364,-0.17881,-0.069823,-0.10839,-0.175991,-0.137414,-0.160571,...,-0.317149,-0.177586,-0.075767,-0.03296,-0.333318,0.393984,-0.04443,-0.283894,-0.302578,-0.017006,-0.014913,-0.014496,-0.016377,-0.015401,-0.471127,-0.241413,-0.178534,-0.106664,-0.585515,-0.183333,-0.021262,-0.021726,-0.159574,-0.14908,-0.154573,-0.324250,-0.445851,-0.038271,-0.117267,-0.216175,-0.342985,6.509144,-0.122832,-0.329040,-0.130763,-0.506981,-0.171434,-0.197585,-0.44033,CD4 T
3,0.080052,0.828744,0.614775,1.051572,-0.552748,0.713069,-0.324617,-0.181656,-0.200979,-0.414643,-0.240882,-0.195287,-0.117689,-0.271599,-0.110263,-0.433555,-0.246179,-0.249721,-0.148742,-0.474248,-0.208137,-0.12555,-0.180029,-0.226363,-0.08094,-0.046267,-0.167078,-0.131427,1.380851,-0.054701,-0.067024,-0.043884,-0.074466,-0.159364,-0.17881,-0.069823,-0.10839,-0.175991,-0.137414,-0.160571,...,3.054919,-0.177586,-0.075767,-0.03296,-0.333318,0.526239,-0.04443,-0.283894,-0.302578,-0.017006,-0.014913,-0.014496,-0.016377,-0.015401,-0.471127,-0.241413,-0.178534,-0.106664,-0.585515,-0.183333,-0.021262,-0.021726,-0.159574,-0.14908,-0.154573,2.931709,-0.445851,-0.038271,-0.117267,4.322459,-0.342985,4.264288,-0.122832,-0.329040,-0.130763,-0.506981,-0.171434,-0.197585,-0.44033,CD4 T
4,0.953832,-0.601956,0.794895,-0.488968,1.413335,-0.614123,-0.324617,-0.181656,-0.200979,-0.414643,-0.240882,-0.195287,-0.117689,-0.271599,-0.110263,-0.433555,-0.246179,-0.249721,-0.148742,1.555869,-0.208137,-0.12555,-0.180029,-0.226363,-0.08094,-0.046267,-0.167078,-0.131427,-0.523544,-0.054701,-0.067024,-0.043884,-0

In [ ]:
newdata.columns[-1]

'celltype.l1.5'

In [ ]:
meta = pd.DataFrame(newdata[newdata.columns[-1]])

In [ ]:
meta.columns = ["perturb"]

In [ ]:
meta

,perturb
0,Progenitor cells
1,gdT
2,CD4 T
3,CD4 T
4,Mono/DC
...,...
30667,CD4 T
30668,CD4 T
30669,Mono/DC
30670,CD4 T


In [ ]:
expr = newdata.drop(newdata.columns[-1], axis  = 1)

In [ ]:
expr

,IGKC,HBA2,HBB,HBA1,IGHA1,IGLC2,JCHAIN,HBM,IGHG1,IGHM,HBD,IGHG3,IGHG2,CA1,PTGDS,IGLC3,MZB1,AHSP,ELANE,HIST1H4C,IGLL1,PRTN3,CA2,MPO,DNTT,HBG2,AZU1,GYPA,PRDX2,HBG1,IGLL5,IGHGP,IGHA2,FAM178B,IGHG4,LYPD2,VPREB1,UBE2C,AL928768.3,APOC1,...,MAP7D2,ARPC1A,FAM46C,SVOPL,TRAV5,TMEM183A,RPL3,LINC00570,BRIX1,DNAJC2,TRBV21-1,LINC00114,RP11-556I14.1,LECT1,SLC16A8,CRIP1,SEL1L3,POLR3H,AMT,SERPINB1,RNF26,RP11-656D10.3,RBP4,MTIF2,INF2,RP11-693J15.5,NAE1,KDELR2,TCEAL2,SPATS2L,INSR,RAD23B,COMMD4,PPARA,PFDN6,PDSS1,BANF1,DDI2,DCAF6,HSPA5
0,-0.428204,0.183748,0.406723,-0.488968,1.248214,-0.614123,-0.324617,-0.181656,-0.200979,-0.414643,2.407124,-0.195287,-0.117689,-0.271599,-0.110263,-0.433555,-0.246179,-0.249721,-0.148742,-0.474248,1.514113,-0.12555,-0.180029,-0.226363,-0.08094,-0.046267,-0.167078,-0.131427,1.081326,-0.054701,-0.067024,-0.043884,-0.074466,-0.159364,-0.17881,-0.069823,-0.10839,2.493712,-0.137414,-0.160571,...,-0.038874,-0.317149,-0.177586,-0.075767,-0.03296,-0.333318,0.739594,-0.04443,-0.283894,2.660236,-0.017006,-0.014913,-0.014496,-0.016377,-0.015401,-0.471127,-0.241413,-0.178534,5.793597,2.086319,-0.183333,-0.021262,-0.021726,-0.159574,-0.14908,-0.154573,-0.324250,-0.445851,-0.038271,-0.117267,-0.216175,2.892558,-0.237605,-0.122832,-0.329040,-0.130763,-0.506981,3.164261,-0.197585,-0.44033
1,-1.047339,-0.601956,-0.985002,-0.488968,-0.552748,-0.614123,-0.324617,-0.181656,4.964708,-0.414643,-0.240882,-0.195287,-0.117689,-0.271599,-0.110263,-0.433555,-0.246179,-0.249721,-0.148742,-0.474248,-0.208137,-0.12555,6.346507,-0.226363,-0.08094,-0.046267,-0.167078,-0.131427,-0.523544,-0.054701,-0.067024,-0.043884,-0.074466,-0.159364,-0.17881,-0.069823,-0.10839,-0.175991,-0.137414,-0.160571,...,-0.038874,-0.317149,-0.177586,-0.075767,-0.03296,-0.333318,1.651260,-0.04443,-0.283894,-0.302578,-0.017006,-0.014913,-0.014496,-0.016377,-0.015401,-0.471127,-0.241413,-0.178534,-0.106664,-0.585515,-0.183333,-0.021262,-0.021726,-0.159574,-0.14908,-0.154573,-0.324250,-0.445851,-0.038271,-0.117267,-0.216175,-0.342985,-0.237605,-0.122832,-0.329040,-0.130763,-0.506981,-0.171434,-0.197585,-0.44033
2,-1.047339,-0.601956,1.534527,-0.488968,-0.552748,-0.614123,-0.324617,-0.181656,-0.200979,-0.414643,-0.240882,-0.195287,-0.117689,-0.271599,-0.110263,-0.433555,-0.246179,-0.249721,-0.148742,-0.474248,-0.208137,-0.12555,-0.180029,-0.226363,-0.08094,-0.046267,-0.167078,-0.131427,2.330472,-0.054701,-0.067024,-0.043884,-0.074466,-0.159364,-0.17881,-0.069823,-0.10839,-0.175991,-0.137414,-0.160571,...,-0.038874,-0.317149,-0.177586,-0.075767,-0.03296,-0.333318,0.393984,-0.04443,-0.283894,-0.302578,-0.017006,-0.014913,-0.014496,-0.016377,-0.015401,-0.471127,-0.241413,-0.178534,-0.106664,-0.585515,-0.183333,-0.021262,-0.021726,-0.159574,-0.14908,-0.154573,-0.324250,-0.445851,-0.038271,-0.117267,-0.216175,-0.342985,6.509144,-0.122832,-0.329040,-0.130763,-0.506981,-0.171434,-0.197585,-0.44033
3,0.080052,0.828744,0.614775,1.051572,-0.552748,0.713069,-0.324617,-0.181656,-0.200979,-0.414643,-0.240882,-0.195287,-0.117689,-0.271599,-0.110263,-0.433555,-0.246179,-0.249721,-0.148742,-0.474248,-0.208137,-0.12555,-0.180029,-0.226363,-0.08094,-0.046267,-0.167078,-0.131427,1.380851,-0.054701,-0.067024,-0.043884,-0.074466,-0.159364,-0.17881,-0.069823,-0.10839,-0.175991,-0.137414,-0.160571,...,-0.038874,3.054919,-0.177586,-0.075767,-0.03296,-0.333318,0.526239,-0.04443,-0.283894,-0.302578,-0.017006,-0.014913,-0.014496,-0.016377,-0.015401,-0.471127,-0.241413,-0.178534,-0.106664,-0.585515,-0.183333,-0.021262,-0.021726,-0.159574,-0.14908,-0.154573,2.931709,-0.445851,-0.038271,-0.117267,4.322459,-0.342985,4.264288,-0.122832,-0.329040,-0.130763,-0.506981,-0.171434,-0.197585,-0.44033
4,0.953832,-0.601956,0.794895,-0.488968,1.413335,-0.614123,-0.324617,-0.181656,-0.200979,-0.414643,-0.240882,-0.195287,-0.117689,-0.271599,-0.110263,-0.433555,-0.246179,-0.249721,-0.148742,1.555869,-0.208137,-0.12555,-0.180029,-0.226363,-0.08094,-0.046267,-0.167078,-0.131427,-0.523544,-0.054701,-0.067024,-0.043884,-0

In [ ]:
expr = expr.transpose()

In [ ]:
expr

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,30632,30633,30634,30635,30636,30637,30638,30639,30640,30641,30642,30643,30644,30645,30646,30647,30648,30649,30650,30651,30652,30653,30654,30655,30656,30657,30658,30659,30660,30661,30662,30663,30664,30665,30666,30667,30668,30669,30670,30671
IGKC,-0.428204,-1.047339,-1.047339,0.080052,0.953832,1.982183,0.091750,-1.047339,1.974582,-1.047339,-1.047339,-1.047339,0.615770,0.236480,-1.047339,-1.047339,-1.047339,0.636896,0.717112,0.484105,0.644306,0.275043,-1.047339,0.138760,-1.047339,0.431264,0.634544,-1.047339,-1.047339,-1.047339,-0.277476,-1.047339,-1.047339,0.073376,0.763909,-1.047339,-1.047339,-1.047339,-1.047339,-1.047339,...,0.577773,1.423658,0.035011,-1.047339,0.008509,2.069583,0.740485,0.324261,-1.047339,-1.047339,0.111533,-1.047339,-0.021941,0.605155,0.425505,-1.047339,-0.372777,0.547710,0.328189,-1.047339,-1.047339,0.471091,-0.295642,-0.485430,-0.553826,0.154187,-0.056224,0.345608,-1.047339,0.633372,-0.035848,1.420359,-0.032248,-1.047339,0.036731,-1.047339,0.201403,-0.160723,0.527459,-1.047339
HBA2,0.183748,-0.601956,-0.601956,0.828744,-0.601956,-0.601956,-0.601956,-0.601956,1.986508,-0.601956,-0.601956,2.485121,-0.601956,-0.601956,0.816080,-0.601956,-0.601956,-0.601956,1.072269,-0.601956,-0.601956,1.076195,-0.601956,0.903247,-0.601956,-0.601956,0.680836,1.461387,1.430192,1.503913,-0.601956,-0.601956,-0.601956,-0.601956,1.126880,-0.601956,1.961798,-0.601956,-0.601956,-0.601956,...,0.911431,1.334623,-0.601956,-0.601956,-0.601956,0.658971,-0.601956,-0.601956,-0.601956,-0.601956,-0.601956,1.031659,-0.601956,-0.601956,0.739097,1.617395,0.759861,0.173073,-0.601956,-0.601956,-0.601956,1.324987,3.796624,0.111126,3.102697,-0.601956,1.172063,-0.601956,-0.601956,-0.601956,-0.601956,1.569027,1.206934,-0.601956,0.773768,0.763410,-0.601956,-0.601956,0.854063,-0.601956
HBB,0.406723,-0.985002,1.534527,0.614775,0.794895,0.735257,0.628270,0.813311,1.407617,0.693205,0.270060,3.013038,0.279269,0.337931,0.603243,0.528512,0.966455,1.528646,0.833206,-0.985002,-0.985002,1.120253,0.103014,0.237234,-0.985002,0.538649,-0.985002,-0.985002,-0.985002,1.711632,2.444574,1.042861,-0.985002,0.882302,-0.985002,1.112255,1.096784,0.670951,1.188875,0.932381,...,0.689622,-0.985002,-0.985002,-0.985002,0.103014,-0.985002,-0.985002,0.012749,0.889329,-0.985002,-0.985002,-0.985002,0.071636,0.031527,0.103943,0.359747,2.906844,-0.985002,-0.985002,-0.985002,-0.985002,-0.985002,4.010042,-0.985002,3.294616,-0.985002,0.036309,-0.985002,0.369420,0.295722,-0.985002,1.264287,0.061016,-0.985002,1.038321,-0.985002,-0.985002,-0.071374,0.197296,0.051135
HBA1,-0.488968,-0.488968,-0.488968,1.051572,-0.488968,-0.488968,-0.488968,0.964731,-0.488968,-0.488968,-0.488968,3.416706,1.187536,-0.488968,-0.488968,1.518048,-0.488968,-0.488968,-0.488968,-0.488968,-0.488968,1.318020,-0.488968,1.131795,1.526422,2.157303,-0.488968,-0.488968,-0.488968,-0.488968,1.647459,-0.488968,-0.488968,-0.488968,1.372597,-0.488968,2.271613,1.118313,-0.488968,-0.488968,...,-0.488968,-0.488968,-0.488968,-0.488968,-0.488968,-0.488968,-0.488968,-0.488968,-0.488968,-0.488968,-0.488968,1.270065,-0.488968,-0.488968,0.955042,-0.488968,2.087435,0.345563,-0.488968,-0.488968,-0.488968,-0.488968,4.345665,-0.488968,3.705516,1.152874,0.865355,1.414445,-0.488968,-0.488968,-0.488968,-0.488968,-0.488968,-0.488968,1.928395,-0.488968,-0.488968,-0.488968,1.078835,0.885016
IGHA1,1.248214,-0.552748,-0.552748,-0.552748,1.413335,-0.552748,-0.552748,-0.552748,1.856992,-0.552748,-0.552748,-0.552748,-0.552748,-0.552748,-0.552748,1.694044,1.648073,-0.552748,-0.552748,-0.552748,-0.552748,-0.552748,1.062398,-0.552748,-0.552748,-0.552748,0.993548,-0.552748,-0.552748,-0.552748,-0.125020,1.753878,-0.552748,-0.552748,-0.552748,-0.552748,-0.552748,-0.552748,-0.552748,-0.552748,...,-0.552748,-0.552748,-0.552748,-0.552748,-0.552748,0.967191,-0.552748,0.928401,-0.552748,-0.552748,1.219995,-0.552748,-0.552748,0.956276,1.063776,1.44351

## Seperate train and test 

In [ ]:
expr.shape[1] * 0.8

24537.600000000002

In [ ]:
train_expr = expr.iloc[:, 0:int(expr.shape[1] * 0.8)]

In [ ]:
test_expr = expr.iloc[:,int(expr.shape[1] * 0.8):-1]

In [ ]:
train_expr

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,24497,24498,24499,24500,24501,24502,24503,24504,24505,24506,24507,24508,24509,24510,24511,24512,24513,24514,24515,24516,24517,24518,24519,24520,24521,24522,24523,24524,24525,24526,24527,24528,24529,24530,24531,24532,24533,24534,24535,24536
IGKC,-0.428204,-1.047339,-1.047339,0.080052,0.953832,1.982183,0.091750,-1.047339,1.974582,-1.047339,-1.047339,-1.047339,0.615770,0.236480,-1.047339,-1.047339,-1.047339,0.636896,0.717112,0.484105,0.644306,0.275043,-1.047339,0.138760,-1.047339,0.431264,0.634544,-1.047339,-1.047339,-1.047339,-0.277476,-1.047339,-1.047339,0.073376,0.763909,-1.047339,-1.047339,-1.047339,-1.047339,-1.047339,...,0.620051,-1.047339,-1.047339,0.797837,0.187603,0.223675,-1.047339,-1.047339,-1.047339,0.648196,-1.047339,-1.047339,-1.047339,-1.047339,-1.047339,-1.047339,-1.047339,-1.047339,0.653926,0.183497,0.932876,0.916702,-0.228201,-0.154267,-1.047339,0.642222,1.674594,0.017601,0.063536,1.127408,-1.047339,2.284746,-1.047339,-1.047339,0.153942,0.004566,0.685336,0.576974,-1.047339,-1.047339
HBA2,0.183748,-0.601956,-0.601956,0.828744,-0.601956,-0.601956,-0.601956,-0.601956,1.986508,-0.601956,-0.601956,2.485121,-0.601956,-0.601956,0.816080,-0.601956,-0.601956,-0.601956,1.072269,-0.601956,-0.601956,1.076195,-0.601956,0.903247,-0.601956,-0.601956,0.680836,1.461387,1.430192,1.503913,-0.601956,-0.601956,-0.601956,-0.601956,1.126880,-0.601956,1.961798,-0.601956,-0.601956,-0.601956,...,-0.601956,-0.601956,-0.601956,-0.601956,-0.601956,-0.601956,5.771971,-0.601956,-0.601956,-0.601956,-0.601956,-0.601956,-0.601956,-0.601956,-0.601956,-0.601956,1.169800,-0.601956,-0.601956,0.748405,-0.601956,-0.601956,0.437560,0.531384,-0.601956,-0.601956,-0.601956,-0.601956,-0.601956,1.001097,0.814776,1.008871,-0.601956,-0.601956,-0.601956,-0.601956,1.596871,-0.601956,-0.601956,0.747630
HBB,0.406723,-0.985002,1.534527,0.614775,0.794895,0.735257,0.628270,0.813311,1.407617,0.693205,0.270060,3.013038,0.279269,0.337931,0.603243,0.528512,0.966455,1.528646,0.833206,-0.985002,-0.985002,1.120253,0.103014,0.237234,-0.985002,0.538649,-0.985002,-0.985002,-0.985002,1.711632,2.444574,1.042861,-0.985002,0.882302,-0.985002,1.112255,1.096784,0.670951,1.188875,0.932381,...,-0.985002,-0.985002,-0.985002,0.451114,-0.985002,0.324736,5.130591,-0.985002,1.257652,-0.985002,-0.985002,0.672819,-0.985002,0.444938,-0.985002,-0.985002,-0.985002,0.171997,-0.985002,-0.111234,0.899440,-0.985002,-0.985002,-0.064722,-0.985002,-0.985002,-0.985002,0.112383,-0.985002,-0.985002,-0.985002,-0.985002,-0.985002,-0.985002,-0.985002,-0.985002,-0.985002,1.170394,0.175439,-0.985002
HBA1,-0.488968,-0.488968,-0.488968,1.051572,-0.488968,-0.488968,-0.488968,0.964731,-0.488968,-0.488968,-0.488968,3.416706,1.187536,-0.488968,-0.488968,1.518048,-0.488968,-0.488968,-0.488968,-0.488968,-0.488968,1.318020,-0.488968,1.131795,1.526422,2.157303,-0.488968,-0.488968,-0.488968,-0.488968,1.647459,-0.488968,-0.488968,-0.488968,1.372597,-0.488968,2.271613,1.118313,-0.488968,-0.488968,...,-0.488968,-0.488968,-0.488968,-0.488968,-0.488968,-0.488968,6.129867,-0.488968,1.840318,-0.488968,0.931584,-0.488968,-0.488968,-0.488968,-0.488968,-0.488968,-0.488968,-0.488968,-0.488968,0.249183,0.842229,1.822198,-0.488968,-0.488968,-0.488968,-0.488968,-0.488968,-0.488968,1.029003,-0.488968,1.036532,-0.488968,-0.488968,0.963480,-0.488968,-0.488968,-0.488968,-0.488968,-0.488968,-0.488968
IGHA1,1.248214,-0.552748,-0.552748,-0.552748,1.413335,-0.552748,-0.552748,-0.552748,1.856992,-0.552748,-0.552748,-0.552748,-0.552748,-0.552748,-0.552748,1.694044,1.648073,-0.552748,-0.552748,-0.552748,-0.552748,-0.552748,1.062398,-0.552748,-0.552748,-0.552748,0.993548,-0.552748,-0.552748,-0.552748,-0.125020,1.753878,-0.552748,-0.552748,-0.552748,-0.552748,-0.552748,-0.552748,-0.552748,-0.552748,...,-0.552748,-0.552748,-0.552748,-0.552748,-0.552748,1.391540,0.312377,-0.552748,-0.552748,-0.552748,1.037515,-0.552748,-0.552748,0.949

In [ ]:
train_meta = meta.iloc[0:int(expr.shape[1] * 0.8),:]

In [ ]:
test_meta = meta.iloc[int(expr.shape[1] * 0.8):-1,:]

In [ ]:
train_meta

,perturb
0,Progenitor cells
1,gdT
2,CD4 T
3,CD4 T
4,Mono/DC
...,...
24532,CD8 T
24533,Mono/DC
24534,Mono/DC
24535,CD4 T


## Augmentation

In [57]:
class DINO_Crop(object):
  def __init__(self, global_crops_scale, local_crops_scale, local_crops_number):
    self.global_crops_scale = global_crops_scale
    self.local_crops_scale = local_crops_scale
    self.local_crops_number = local_crops_number
    
  def __call__(self, x):
    inputs = []
    data = x  
    length = x.shape[0]
    print("This is the length")
    print(length)
    global_size = int(length * self.global_crops_scale)
    global_index_1 = np.random.choice(length, size = global_size, replace = False)
    
    global_corr_1 = torch.from_numpy(global_index_1)
    
    global_data_1 = data[global_index_1,]
    global_input_1 = torch.cat([global_data_1, global_corr_1]).float()
    #y = global_input_1.numpy().tolist()
    #global_input_1 = torch.tensor([2*(i-min(y))/(max(y)-min(y))-1 for i in y])
    
    return global_input_1

In [ ]:
perturb_list = ["A2M", "CCND1"]

In [ ]:
perturb_list = ["0", "1"]

In [59]:
crop = DINO_Crop(0.5,0.25,8)

# Load perturbed data

In [ ]:
perturb_list = ["A2M", "CCND1", "DAG1", "FBL", "IDH2", "IFNGR1","JAK1", "MIA", "PIK3IP1", "SLC19A1"]
   

In [ ]:
expression_matrix_path = "/content/drive/MyDrive/scRNA/subset_hvg/train"

In [ ]:
test_path = "/content/drive/MyDrive/scRNA/subset_hvg/test"

In [ ]:
expr_train = pd.DataFrame()
for number_lab, label in zip(range(len(perturb_list)),perturb_list):
      path = expression_matrix_path + "/" + label + ".csv"
      df = pd.read_csv(path, index_col = 0)
      if expr_train.shape[0] == 0:
        expr_train = df
      else:
        expr_train = pd.merge(expr_train, df, how = "outer", left_index = True, right_index = True)

In [ ]:
expr_test = pd.DataFrame()
for number_lab, label in zip(range(len(perturb_list)),perturb_list):
      path = test_path + "/" + label + ".csv"
      df = pd.read_csv(path, index_col = 0)
      if expr_test.shape[0] == 0:
        expr_test = df
      else:
        expr_test = pd.merge(expr_test, df, how = "outer", left_index = True, right_index = True)

In [ ]:
expr

,CELL_42982,CELL_9744,CELL_79203,CELL_93928,CELL_61444,CELL_49466,CELL_112865,CELL_142647,CELL_67074,CELL_106938,CELL_91884,CELL_64497,CELL_43505,CELL_123202,CELL_35750,CELL_149245,CELL_38432,CELL_182955,CELL_49138,CELL_176320,CELL_214174,CELL_157517,CELL_43774,CELL_71660,CELL_33939,CELL_107191,CELL_74837,CELL_82174,CELL_140322,CELL_99741,CELL_34110,CELL_172306,CELL_178336,CELL_109160,CELL_59296,CELL_182363,CELL_218281,CELL_112000,CELL_212493,CELL_22866,...,CELL_66848,CELL_154867,CELL_83375,CELL_44070,CELL_139445,CELL_134105,CELL_189449,CELL_19189,CELL_177545,CELL_78446,CELL_39993,CELL_132846,CELL_65225,CELL_50563,CELL_166744,CELL_192818,CELL_190979,CELL_207914,CELL_175525,CELL_147421,CELL_127596,CELL_137229,CELL_189837,CELL_138707,CELL_44158,CELL_118808,CELL_197539,CELL_31012,CELL_21858,CELL_146471,CELL_6331,CELL_216883,CELL_115175,CELL_112201,CELL_131759,CELL_82183,CELL_167090,CELL_84014,CELL_95816,CELL_183125
GBP1,5.456098,0.000000,8.104459,7.898718,7.446193,0.000000,7.636912,6.488857,7.507961,7.307319,8.768065,6.971482,4.645079,5.370616,0.000000,5.096223,0.000000,5.946741,0.000000,6.470244,4.976297,5.769317,4.979375,7.495771,0.000000,8.258555,7.110199,7.480274,7.992248,7.125577,4.672457,4.876786,5.005234,7.755065,7.252990,0.000000,6.333411,7.439712,7.164970,0.000000,...,7.334586,6.716901,8.066916,4.672195,7.191382,5.642458,6.328165,0.000000,4.773008,7.293984,0.000000,7.804651,8.197011,0.00000,5.682258,6.380295,0.000000,4.554277,4.856453,4.919028,8.261949,8.059346,5.218244,8.181001,0.000000,7.921586,0.000000,0.000000,0.000000,6.476000,5.360550,6.489609,7.941800,8.240432,6.545245,6.146759,6.286982,7.854893,8.000500,6.114101
HLA-B,4.767211,0.000000,6.832270,5.264474,6.061648,4.952221,6.401328,0.000000,6.866601,5.923035,5.853014,6.125434,0.000000,6.753416,0.000000,8.268394,4.533835,8.246970,5.158857,8.078443,7.644001,7.307306,5.669077,6.749174,0.000000,6.362902,4.814938,5.924243,6.606967,4.649478,5.583123,7.702801,8.256865,5.678619,6.665770,7.797523,8.164499,0.000000,8.445346,5.482688,...,6.775460,7.932444,5.850282,4.672195,5.727551,6.651802,7.307877,4.972611,8.198780,5.829910,0.000000,5.505730,6.776490,5.15260,8.482413,7.765317,7.921700,7.316952,8.250103,8.313138,6.066787,6.451172,7.898761,5.880932,0.000000,5.440662,8.015108,5.738992,0.000000,7.535865,4.672090,7.226415,6.476647,6.122100,6.322461,5.455750,7.266648,6.710592,4.713343,7.990997
WARS,4.767211,0.000000,6.678299,6.359633,6.848834,4.952221,7.147668,7.075968,7.271718,6.837718,6.949710,6.635385,0.000000,7.669006,0.000000,5.096223,4.533835,6.533365,0.000000,6.183078,7.272653,6.460902,4.979375,6.854417,0.000000,7.614433,7.372375,6.328816,7.011982,7.125577,0.000000,5.970305,6.791393,7.621595,6.442945,6.189726,6.556200,7.216715,5.558621,0.000000,...,7.585756,0.000000,5.850282,0.000000,6.929244,7.585326,6.482060,0.000000,7.473136,7.127054,5.724707,6.196844,7.228061,0.00000,6.596503,6.093177,6.536494,4.554277,5.545703,6.522604,7.269135,7.431014,6.823340,6.795546,4.213937,7.228801,6.917157,4.142344,0.000000,6.225125,0.000000,5.480663,7.456514,7.586749,7.237673,5.455750,7.059170,6.710592,7.007818,6.114101
STAT1,0.000000,0.000000,7.658342,6.869766,7.158706,0.000000,7.294164,0.000000,7.048749,7.479063,6.949710,7.222588,4.645079,7.764274,0.000000,6.344605,0.000000,6.282466,4.471442,5.778645,6.474993,6.374032,4.979375,7.259530,5.316326,7.278158,7.205435,7.369115,6.829841,7.348559,5.360919,4.876786,5.005234,7.062346,7.135296,4.588457,7.297392,7.621935,7.164970,0.000000,...,7.468036,6.583542,6.804020,4.672195,6.237074,7.687060,6.146200,3.887754,7.250134,7.213998,0.000000,6.601630,7.228061,0.00000,6.240413,6.603099,6.536494,5.242149,5.949866,6.522604,6.912759,7.654039,6.313239,7.488133,4.213937,7.228801,6.447747,0.000000,4.670413,6.986209,0.000000,6.924390,7.774811,6.968144,6.322461,7.243943,6.105032,7.403130,6.651531,6.850546
MT2A,5.169838,8.285876,6.832270,8.478373,8.868858,7.248425,8.856545,9.512701,8.029035,7.425028,7.555410,7.664159,4.645079,8.921435,7.272294,6.962833,7.296304,5.

In [ ]:
meta = pd.read_csv("/content/drive/MyDrive/scRNA/data/RNA_metadata.csv", index_col= 0)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
cell_list_train = expr_train.columns

In [ ]:
cell_list_test = expr_test.columns

In [ ]:
cell_list_test

Index(['CELL_3589', 'CELL_4253', 'CELL_6039', 'CELL_9591', 'CELL_12240',
       'CELL_14575', 'CELL_18115', 'CELL_18760', 'CELL_19023', 'CELL_21543',
       ...
       'CELL_194871', 'CELL_197364', 'CELL_199181', 'CELL_200244',
       'CELL_207279', 'CELL_209925', 'CELL_211635', 'CELL_214559',
       'CELL_215376', 'CELL_218325'],
      dtype='object', length=999)

In [ ]:
meta_train = meta.loc[cell_list_train]

In [ ]:
meta_test = meta.loc[cell_list_test]

In [ ]:
meta_train

,library_preparation_protocol,condition,MOI,sgRNA,UMI_count
CELL_42982,10X 3' v3 sequencing,Control,1,A2M_2,17154.0
CELL_9744,10X 3' v3 sequencing,Control,1,A2M_2,13110.0
CELL_79203,10X 3' v3 sequencing,IFNγ,1,A2M_1,7557.0
CELL_93928,10X 3' v3 sequencing,IFNγ,1,A2M_3,5199.0
CELL_61444,10X 3' v3 sequencing,IFNγ,1,A2M_1,11680.0
...,...,...,...,...,...
CELL_82183,10X 3' v3 sequencing,IFNγ,1,SLC19A1_2,4290.0
CELL_167090,10X 3' v3 sequencing,Co-culture,1,SLC19A1_2,11183
CELL_84014,10X 3' v3 sequencing,IFNγ,1,SLC19A1_3,8536.0
CELL_95816,10X 3' v3 sequencing,IFNγ,1,SLC19A1_3,9056.0


In [ ]:
meta_test

,library_preparation_protocol,condition,MOI,sgRNA,UMI_count
CELL_3589,10X 3' v3 sequencing,Control,1,A2M_3,19964.0
CELL_4253,10X 3' v3 sequencing,Control,1,A2M_2,9278.0
CELL_6039,10X 3' v3 sequencing,Control,1,A2M_2,8032.0
CELL_9591,10X 3' v3 sequencing,Control,1,A2M_1,8117.0
CELL_12240,10X 3' v3 sequencing,Control,1,A2M_2,10795.0
...,...,...,...,...,...
CELL_209925,10X 3' v3 sequencing,Co-culture,1,SLC19A1_2,16133
CELL_211635,10X 3' v3 sequencing,Co-culture,1,SLC19A1_3,32690
CELL_214559,10X 3' v3 sequencing,Co-culture,1,SLC19A1_3,17864
CELL_215376,10X 3' v3 sequencing,Co-culture,1,SLC19A1_1,14684


In [ ]:
meta_train["perturb"] = meta_train.apply(lambda x: x.sgRNA.split("_")[0], axis = 1)

In [ ]:
meta_test["perturb"] = meta_test.apply(lambda x: x.sgRNA.split("_")[0], axis = 1)

In [ ]:
meta_test

,library_preparation_protocol,condition,MOI,sgRNA,UMI_count,perturb
CELL_3589,10X 3' v3 sequencing,Control,1,A2M_3,19964.0,A2M
CELL_4253,10X 3' v3 sequencing,Control,1,A2M_2,9278.0,A2M
CELL_6039,10X 3' v3 sequencing,Control,1,A2M_2,8032.0,A2M
CELL_9591,10X 3' v3 sequencing,Control,1,A2M_1,8117.0,A2M
CELL_12240,10X 3' v3 sequencing,Control,1,A2M_2,10795.0,A2M
...,...,...,...,...,...,...
CELL_209925,10X 3' v3 sequencing,Co-culture,1,SLC19A1_2,16133,SLC19A1
CELL_211635,10X 3' v3 sequencing,Co-culture,1,SLC19A1_3,32690,SLC19A1
CELL_214559,10X 3' v3 sequencing,Co-culture,1,SLC19A1_3,17864,SLC19A1
CELL_215376,10X 3' v3 sequencing,Co-culture,1,SLC19A1_1,14684,SLC19A1


## Train dataset 

In [60]:
train_dataset = scRNACSV(expr_train, meta_train, transform = crop)

This is the label dictionary of this dataset {'A2M': 0, 'CCND1': 1, 'DAG1': 2, 'FBL': 3, 'IDH2': 4, 'IFNGR1': 5, 'JAK1': 6, 'MIA': 7, 'PIK3IP1': 8, 'SLC19A1': 9}


In [61]:
x, y = train_dataset[503]

This is the length
2000


In [62]:
x.shape

torch.Size([2000])

In [ ]:
x

tensor([-0.9951, -0.9947, -1.0000,  ..., -0.9890, -0.9680,  0.1436])

In [ ]:
y

1

In [ ]:
#sampler = torch.utils.data.DistributedSampler(train_dataset, shuffle=True)
    
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle = True)

In [ ]:
test_path = "/content/drive/MyDrive/scRNA/pseudo_data/test"

## Test dataset

In [ ]:
test_dataset = scRNACSV(expr_test, meta_test, transform = crop)

This is the label dictionary of this dataset {'A2M': 0, 'CCND1': 1, 'DAG1': 2, 'FBL': 3, 'IDH2': 4, 'IFNGR1': 5, 'JAK1': 6, 'MIA': 7, 'PIK3IP1': 8, 'SLC19A1': 9}


In [ ]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle = True)

In [ ]:
for i, data in enumerate(train_loader, 0):
  inputs, labels = data
  print(inputs.shape)
  print(labels)
  break

torch.Size([16, 2000])
tensor([6, 4, 5, 8, 0, 8, 8, 6, 5, 5, 0, 2, 0, 9, 5, 7])


# Prepare Model

In [56]:
class VisionTransformer(nn.Module):
  def __init__(self, img_size=[224],in_chans=1, num_classes=10, embed_dim=256, mlp_ratio=4., depth = 1, qkv_bias=False, qk_scale=None, drop_rate=0., attn_drop_rate=0.,
                 drop_path_rate=0., norm_layer=nn.LayerNorm, **kwargs):
    super().__init__()

    # Number of features at the end of this module
    self.num_features = self.embed_dim = embed_dim # =3 in this golden truth modle
    
    # Initiate the [CLS] token and make it not learnable
    self.cls_token = nn.Parameter(torch.from_numpy(-2* np.ones((1,1,embed_dim)))).float()
    #self.cls_token = nn.Parameter(torch.from_numpy(np.array([[[0.0,-1.0,-1.0]]])).float())
    self.pos_drop = nn.Dropout(p=drop_rate)
    
    # Set SDG?
    #dpr = [x.item() for x in torch.linspace(0, drop_path_rate, 1)]  # stochastic depth decay rule
    
    # Set Norm layer
    self.norm = norm_layer(embed_dim)

    # Set the blk 
    dpr = [x.item() for x in torch.linspace(0, drop_path_rate, depth)]  # stochastic depth decay rule
    self.blocks = nn.ModuleList([
            Block(
                dim=embed_dim, num_heads= 1, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[i], norm_layer=norm_layer)
            for i in range(depth)])
    # This head???
    self.head = nn.Linear(embed_dim, num_classes)
    self.Embedding = nn.Embedding(2000, 128)
    # SoftMax
    #self.softMax = F.log_softmax(10)
    #trunc_normal_(self.cls_token, std=.02)
    self.apply(self._init_weights) #???
    self.act = nn.LeakyReLU()
    self.exprProj = nn.Linear(1,128)
    
    

  def _init_weights(self, m):
    if isinstance(m, nn.Linear):
      trunc_normal_(m.weight, std=.02)
      if isinstance(m, nn.Linear) and m.bias is not None:
        nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.LayerNorm):
      nn.init.constant_(m.bias, 0)
      nn.init.constant_(m.weight, 1.0)

  def prepare_tokens(self, x):
    B, L = x.shape
    x = self.ReformatInput_cat(x)
    # add the [CLS] token to the embed patch tokens
    cls_tokens = self.cls_token.expand(B, -1, -1)#.cuda()
    x = torch.cat((cls_tokens, x), dim=1)
    # print(f"Here is the device of x {x.device}")
    #print(f"Here is the device of cls_tokens {cls_token.device}")
    return self.pos_drop(x)
  def ReformatInput_cat(self, x):
    # Split the index part and the expression data part
    # x dimension now B x G*2 (B is the batch size, G is the number of genes sampled)
    B, G_2 = x.shape
    G = int(G_2/2)
    expr, index = torch.split(x, (G,G), dim = 1)
    # expr and index are both B x G now
    #print(expr.shape)
    geneEmbedding = self.Embedding(index.int())
    #geneEmbedding = index.reshape(B, G, 1)
  
    # geneEmbedding is B x G x Embed_dim
    #print(geneEmbedding.shape)
    # Reshape the expr to add one dimension to match the geneEmbedding
    expr = expr.reshape(B, G, 1)
    #print("Here is the ")
    expr = self.exprProj(expr)
  
    # cat the expr and gene Embedding
    x = torch.cat((expr, geneEmbedding), dim = 2)
    y = x.numpy().tolist()
    x = torch.tensor([2*(i-min(y))/(max(y)-min(y))-1 for i in y])
    # print(x.shape)
    return x
  def forward(self, x):
    #print(x.shape)
    B, L = x.shape
    x = self.prepare_tokens(x)
    #print(x.grad)
    #print("Here is the original data:")
    #print(x[0:3])
    for blk in self.blocks:
      x = blk(x)
    x = self.norm(x)
    #print("Here is the data before mean:")
    #print(x[0:3])
    #x = torch.mean(x, dim = 1).reshape(B,3)
    #print("Here is the output before the projection:")
    #print(x[0:3])
    # Use class token
    x = self.head(x[:,0])
    #print(x.shape)
    x = self.act(x)
    #x = self.head(x)
    #print(x.shape)
    #x = x.softmax(dim = -1)
    #print(x.grad)
    return x
  def get_last_selfattention(self, x):
        x = self.prepare_tokens(x)
        for i, blk in enumerate(self.blocks):
            if i < len(self.blocks) - 1:
                x = blk(x)
            else:
                # return attention of the last block
                return blk(x, return_attention=True)

  def get_intermediate_layers(self, x, n=1):
        x = self.prepare_tokens(x)
        # we return the output tokens from the `n` last blocks
        output = []
        for i, blk in enumerate(self.blocks):
            x = blk(x)
            if len(self.blocks) - i <= n:
                output.append(self.norm(x))
        return output

In [ ]:
def vit_tiny(patch_size=16):
    model = VisionTransformer(
        patch_size=patch_size, embed_dim=256, depth=3, num_heads=1, mlp_ratio=4,
        qkv_bias=True, norm_layer=partial(nn.LayerNorm, eps=1e-6))
    return model

## Drop

In [ ]:
def drop_path(x, drop_prob: float = 0., training: bool = False):
    if drop_prob == 0. or not training:
        return x
    keep_prob = 1 - drop_prob
    shape = (x.shape[0],) + (1,) * (x.ndim - 1)  # work with diff dim tensors, not just 2D ConvNets
    random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
    random_tensor.floor_()  # binarize
    output = x.div(keep_prob) * random_tensor
    return output


class DropPath(nn.Module):
    """Drop paths (Stochastic Depth) per sample  (when applied in main path of residual blocks).
    """
    def __init__(self, drop_prob=None):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        return drop_path(x, self.drop_prob, self.training)


## Block


In [ ]:
class Block(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = Attention(
            dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

    def forward(self, x, return_attention=False):
        y, attn = self.attn(self.norm1(x))
        if return_attention:
            return attn
        x = x + self.drop_path(y)
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x

## Attention

In [ ]:
class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x, attn

## MLP

In [ ]:
class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

# Train one Epoch

In [ ]:
def train_one_epoch(trainloader, simple_model, epoch, optimizer):
  running_loss = 0.0
  simple_model.train()
  lossF = nn.CrossEntropyLoss()
  sum_loss = 0
  num = 0
  for batch_idx, (data, label) in enumerate(trainloader):
    #data, label = Variable(data).cuda(), Variable(label).cuda()
    data, label = Variable(data), Variable(label)
    optimizer.zero_grad()
    output = simple_model(data)
    loss = lossF(output, label)
    sum_loss += loss
    num += 1
    loss.backward()
    optimizer.step()
    plot_grad_flow(simple_model.named_parameters())
    if batch_idx % 100 == 0:
      print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss {:.6f}".format(
                epoch, batch_idx*len(data), len(trainloader.dataset),
                100.*batch_idx/len(trainloader), loss.item()))
  loss_stat = (sum_loss/num).detach().cpu().numpy().item()
  log_stats = {**{f'Average Loss': loss_stat}, 'epoch': epoch + 1}
  with (Path(output_dir) / "loss.txt").open("a") as f: f.write(json.dumps(log_stats) + "\n")

# Train

In [ ]:
os.chdir("/content/drive/MyDrive/scRNA/Supervised_results")

In [ ]:
!rm -r ./128+128_perturb

In [50]:
output_dir = './128+128_perturb' ### customized your own output_dir
model = vit_tiny()
optimizer = optim.Adam(model.parameters(), lr = 1e-4)
#model.cuda()
Path(output_dir).mkdir(parents=True, exist_ok=True)
saveckp_freq = 1
to_restore = {"epoch": 0}
utils.restart_from_checkpoint(
    os.path.join(output_dir, "checkpoint.pth"),
    run_variables=to_restore,
    model=model,
    optimizer=optimizer,
)
start_epoch = to_restore["epoch"]
for epoch in range(start_epoch, 150):
    train_one_epoch(train_loader, model, epoch, optimizer)
    # ============ writing logs ... ============
    save_dict = {
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'epoch': epoch + 1,
    }
    # Train loop
    if epoch % 10 == 0:
      correct, total = 0, 0
    
      with torch.no_grad():
        # Iterate over the test data and generate predictions
          for i, data in enumerate(train_loader, 0):
          # Get inputs
            inputs, targets = data
            inputs = inputs#.cuda()
            targets = targets#.cuda()
          # Generate outputs
            model.eval()
            outputs = model(inputs)
          # Set total and correct
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            # Print accuracy
          train_acc = "%f %%" %(100 * correct / total)
          print(train_acc)
    # Testing loop
      correct, total = 0, 0
      with torch.no_grad():
        # Iterate over the test data and generate predictions
          for i, data in enumerate(test_loader, 0):
          # Get inputs
            inputs, targets = data
            inputs = inputs#.cuda()
            targets = targets#.cuda()
          # Generate outputs
            model.eval()
            outputs = model(inputs)
          # Set total and correct
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
        # Print accuracy
          test_acc =  "%f %%" %(100 * correct / total)
          print(test_acc)
      acc_stats = {"Training Accuracy": train_acc, "Testing Accuracy":  test_acc, "epoch": epoch+1}
      with (Path(output_dir) / "acc.txt").open("a") as f: f.write(json.dumps(acc_stats) + "\n")
    utils.save_on_master(save_dict, os.path.join(output_dir, 'checkpoint.pth'))
    if saveckp_freq and epoch % saveckp_freq == 0:
        utils.save_on_master(save_dict, os.path.join(output_dir, f'checkpoint{epoch:04}.pth'))

IndexError: ignored